In [43]:
import altair as alt
import numpy as np
import polars as pl
import pandas as pd
import pyarrow
from vega_datasets import data
import datetime as dt


#Data Analysis of AIAN Death Rates Where Tribal Authorities Do and Do not Have Criminal Jurisdiction to Prosecute Minor Crimes

In [44]:
AIAN_deaths = pl.read_csv("data/AIAN_drds_state.csv")

# Need to make sure our FIPS codes are read as strings
def pad_code(int):
    string = str(int)
    #if len(string) < 2:
    #   string = "".join(["0", string])
    
    return string

def make_juris_labels(int):
    out = "Unknown"
    if int == 1:
        out = "State"
    if int == 0:
        out = "Tribal Authority"
    
    return out

AIAN_deaths = AIAN_deaths.with_columns((pl.col("OD_rate_AIAN")*100000).round(decimals = 3).alias("drd_AIAN_p100k"))
AIAN_deaths = AIAN_deaths.with_columns((pl.col("OD_rate_non_AIAN")*100000).round(decimals = 3).alias("drd_non_p100k"))
AIAN_deaths = AIAN_deaths.with_columns(pl.col("state_code").map_elements(pad_code).alias("id"))
AIAN_deaths = AIAN_deaths.with_columns(pl.col("state_crim").map_elements(make_juris_labels
                                                                         ).alias("Jurisdiction"))

centroids = pl.read_csv("data/world_country_and_usa_states_latitude_and_longitude_values.csv")
centroids = centroids.filter(pl.col("usa_state_latitude").is_not_null())
centroids = centroids.with_columns(pl.col("usa_state_code").alias("State"),
                                   pl.col("usa_state_latitude").alias("latitude"),
                                   pl.col("usa_state_longitude").alias("longitude"))
centroids = centroids["State", "latitude", "longitude"]
AIAN_deaths = AIAN_deaths.join(centroids, "State", "left")


/var/folders/p9/6789_cm149s8kj127lfbgwy40000gn/T/ipykernel_38282/3260050049.py:22: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  AIAN_deaths = AIAN_deaths.with_columns(pl.col("state_code").map_elements(pad_code).alias("id"))
/var/folders/p9/6789_cm149s8kj127lfbgwy40000gn/T/ipykernel_38282/3260050049.py:23: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  AIAN_deaths = AIAN_deaths.with_columns(pl.col("state_crim").map_elements(make_juris_labels


In [45]:
# First we load the data
def group_states():

    grouped_df = AIAN_deaths.group_by("State").agg(pl.col("population_AIAN").sum(),
                                                   pl.col("deaths_AIAN").sum(),
                                                   pl.col("Jurisdiction").first(),
                                                   pl.col("id").first(),
                                                   pl.col("latitude").first(),
                                                   pl.col("longitude").first(),
                                                   pl.col("pop_non_AIAN").sum(),
                                                   pl.col("deaths_non_AIAN").sum())
    
    grouped_df = grouped_df.with_columns((100000*pl.col("deaths_AIAN")/pl.col("population_AIAN")
                                        ).alias("drd_AIAN_p100k"),
                                        (100000*pl.col("deaths_non_AIAN")/pl.col("pop_non_AIAN")
                                        ).alias("drd_non_p100k"))

    return grouped_df

def group_years(df = AIAN_deaths):

    grouped_df = df.group_by("Year", "Jurisdiction").agg(pl.col("population_AIAN").sum(),
                                                   pl.col("deaths_AIAN").sum(),
                                                   pl.col("pop_non_AIAN").sum(),
                                                   pl.col("deaths_non_AIAN").sum())
    
    grouped_df = grouped_df.with_columns((100000*pl.col("deaths_AIAN")/pl.col("population_AIAN")
                                          ).alias("drd_AIAN_p100k"),
                                          (100000*pl.col("deaths_non_AIAN")/pl.col("pop_non_AIAN")
                                          ).alias("drd_non_p100k"))

    return grouped_df

def group_all_years():

    grouped_df = AIAN_deaths.group_by("Year").agg(pl.col("population_AIAN").sum(),
                                                   pl.col("deaths_AIAN").sum(),
                                                   pl.col("pop_non_AIAN").sum(),
                                                   pl.col("deaths_non_AIAN").sum())
    
    grouped_df = grouped_df.with_columns((100000*pl.col("deaths_AIAN")/pl.col("population_AIAN")
                                          ).alias("drd_AIAN_p100k"),
                                          (100000*pl.col("deaths_non_AIAN")/pl.col("pop_non_AIAN")
                                          ).alias("drd_non_p100k"))

    return grouped_df

def pivot_long():
    df = pl.read_csv("data/AIAN_drds_state.csv")
    df = df["Jurisdiction", "State", "Year", "OD_rate_AIAN"]
    df.pivot(columns = "state_crim", index=["State", "Year"], values = "OD_rate_AIAN")
    return df


In [46]:
def map_states():
    df = group_states()
    df = df.to_pandas()
    states = alt.topo_feature(data.us_10m.url, 'states')
    default_value = "No Recognized Tribal Lands"

    full_map = alt.Chart(states).mark_geoshape().encode(
        color=  alt.condition(
            alt.datum.value == default_value,
            alt.value('white'),  # Default color
            alt.Color('Jurisdiction:N', scale=alt.Scale(
            domain = ["State", "Tribal Authority", "No Recognized Tribal Lands"],
            range = ["orange", "lightblue", "wheat"]
            ))
        )
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(df, 'id', list(df.columns)),
        default = default_value
    ).properties(
        width=500,
        height=300
    ).project(
        type='albersUsa'
    )

    return full_map

# Figuring out the bubble overlay required AI, I asked:
# 1) How can I overlay bubbles onto a map of U.S. states that I made in altair?
# 2) What if I don't have the centroid lattitude and longitude data, 
# is there a public data set that has centroid lattitude and longitude data?

def bubble_overlay():
    df = group_states()
    df = df.to_pandas()
    chart = alt.Chart(df).mark_circle().encode(
        longitude = 'longitude:Q',
        latitude = 'latitude:Q',
        size = alt.Size('population_AIAN:Q', title = "Indigenous American Population"),
        color = alt.value("#06402B")
    ).project("albersUsa")
    return chart

map_states().properties(
    title = "Overview of Which States Have Criminal Jurisdiction on Tribal Lands")


alt.Chart(...)

This project is about exploring if on the Native American reservations where tribal authorities
lack criminal jurisdiction to prosecute minor offenses, drug-related deaths are higher. We begin
with a general overview of the states where tribal authorities, as opposed to the state, have jurisdiction.
It is important to note the Indigenous American (or American Indain and Alaskan Native) population 
for each state. State's with smaller populations, have less weight in our analysis.

In [47]:
def ddr_years():
    df = group_all_years()

    chart_AIAN = alt.Chart(df).mark_line(color = "#b048dd").encode(
    x = alt.X('Year:Q', axis=alt.Axis(format='d')),
    y = alt.Y("drd_AIAN_p100k:Q",
                title = "Drug Related Death Rate per 100k persons"))
    
    chart_NON = alt.Chart(df).mark_line(color = "#3dec53").encode(
    x = alt.X('Year:Q', axis=alt.Axis(format='d')),
    y = alt.Y("drd_non_p100k:Q",
                title = ""))
    full = (chart_AIAN + chart_NON).properties(
        title = "Annual Drug Related Death Rate Consistently Increasing Since 1998")
    return full

# This source taught me to use transform fold to when plotting two variables:
#   https://stackoverflow.com/questions/60128774/adding-legend-to-layerd-chart-in-altair 

def ddr_years_fold():
    df = group_all_years().with_columns(pl.col('drd_AIAN_p100k').alias("AIAN"),
                                        pl.col('drd_non_p100k').alias("Non AIAN"))
    chart = alt.Chart(df).mark_line().transform_fold(
        fold=['AIAN', 'Non AIAN'], 
        as_=['Population', 'value']
    ).encode(
        x = alt.X('Year:Q', axis=alt.Axis(format='d')),
        y = alt.Y("value:Q", title = "Drug Related Death Rate per 100k persons"),
        color = alt.Color("Population:N", scale=alt.Scale(
            domain = ["AIAN", "Non AIAN"],
            range = ["#b048dd",  "#3dec53"]
            ))
                          
    ).properties(
        title = "Annual Drug Related Death Rate Consistently Increasing Since 1998")

    return chart

ddr_years_fold()

alt.Chart(...)

Since 1998 the drug-related death rate for American Indian and Alaskan Native (AIAN)
persons, and all other racial groups, has consistently increased year over year.
The main story here, is the continuing proliferation of the opiods epidemic. These 
steady increases also mean that time has to be accounted for when making comparisons.

In [48]:
def ddr_years_faceted():

    df_state_juris = AIAN_deaths.filter(pl.col("Jurisdiction")=="State")
    df_state_juris =df_state_juris.with_columns(pl.col('drd_AIAN_p100k').alias("AIAN"),
                                        pl.col('drd_non_p100k').alias("Non AIAN"))
    
    df_tribe_juris = AIAN_deaths.filter(pl.col("Jurisdiction")== "Tribal Authority")
    df_tribe_juris =df_tribe_juris.with_columns(pl.col('drd_AIAN_p100k').alias("AIAN"),
                                        pl.col('drd_non_p100k').alias("Non AIAN"))
    
    base = alt.Chart().mark_line().transform_fold(
            fold=['AIAN', 'Non AIAN'], 
            as_=['Population', 'value']
        ).encode(
            x = alt.X('Year:Q', axis=alt.Axis(format='d')),
            y = alt.Y("value:Q", title = "Drug Related Death Rate per 100k persons"),
            color = alt.Color("Population:N", scale=alt.Scale(
                domain = ["AIAN", "Non AIAN"],
                range = ["#b048dd",  "#3dec53"]
                )))

    full_state = (base).facet(facet = "State:N",
                        columns = 1, data = df_state_juris,
                        title = "State Has Criminal Jurisdiction")
    
    full_tribe = (base).facet(facet = "State:N",
                    columns = 1, data = df_tribe_juris,
                    title = "Tribal Authority Has Criminal Jurisdiction")
    
    full_chart = alt.hconcat(
    full_state,
    full_tribe
        ).resolve_scale(
            x='shared',
            y='shared'
        )

    return full_chart

ddr_years_faceted()

alt.HConcatChart(...)

In [ ]:
def ddr_years_faceted_new():

    df =AIAN_deaths.with_columns(pl.col('drd_AIAN_p100k').alias("AIAN"),
                                        pl.col('drd_non_p100k').alias("Non AIAN"))
    

    
    base = alt.Chart().mark_line().transform_fold(
            fold=['AIAN', 'Non AIAN'], 
            as_=['Population', 'value']
        ).encode(
            x = alt.X('Year:Q', axis=alt.Axis(format='d')),
            y = alt.Y("value:Q", title = "Drug Related Death Rate per 100k persons"),
            color = alt.Color("Population:N", scale=alt.Scale(
                domain = ["AIAN", "Non AIAN"],
                range = ["#b048dd",  "#3dec53"]
                )))

    facet_chart = (base).facet(facet = "State:N", data = df,
                               columns = 3
            ).properties(
                title = "The Opiods Epidemic Reached States at Different Times"
            )


    return facet_chart

ddr_years_faceted_new()

Breaking out the Drug Related Death Rate over time and splitting up states depending on how
criminal jurisdiction is handled on reservations, no clear trends emerge. It is important to note
that the steady year over year trend that existed when we zoomed out is more irregular at the state level.
Instead many states have a "tipping point" after which the death rate begins to rise more rapidly.

In [49]:
prop_scores = pl.read_csv("data/z_table_states.csv")

def show_z():
    bar = alt.Chart(prop_scores).mark_errorbar().encode(
        alt.Y("CI_upper:Q").scale(zero=False).title(""),
        alt.Y2("CI_lower:Q"),
        alt.X("Jurisdiction:N"),
    )

    point = alt.Chart(prop_scores).mark_point(
        filled=True,
        color="purple"
    ).encode(
        alt.Y("OD_rate_AIAN:Q", title = "Mean AIAN Drug Related Death Rate"),
        alt.X("Jurisdiction:N")
    )

    chart = (point + bar).properties(
        title = "95% Confidence Interval for Average AIAN Drug Related Death Rate"
    )
    return chart

show_z()


alt.LayerChart(...)

Our first look at the different in death rate's is to compare the mean death rate's
and their standard deviations. The formal test for significance is called a z-score.
Here it looks like the relationship is the reverse of what we hypothesized, where
states hold criminal jurisdiction the death rate is a lot lower. Lets see if this
holds when we control for other variables.

In [50]:
# .stack('zero')
def pl280_density():

    df = AIAN_deaths
    density_plot = alt.Chart(df).transform_density(
        'drd_AIAN_p100k',
        counts = False,
        groupby = ["Jurisdiction"],
        as_ = ['AIAN Drug Related Deaths Per 100K', 'Density'],
        bandwidth = 20,
        #extent = [0, 400],
        #steps = 200
    ).mark_area(
        opacity = 0.5
    ).encode(
    x=alt.X("AIAN Drug Related Deaths Per 100K:Q").title("AIAN Drug Related Deaths Per 100K"),
    y= alt.Y('Density:Q').stack(None),
    color = alt.Color('Jurisdiction:N', scale=alt.Scale(
            domain = ["State", "Tribal Authority"],
            range = ["#FFAE42", "#417ecf"])
    ))

    lines = alt.Chart(df).mark_rule(size=2).encode(
    x=alt.X('mean(drd_AIAN_p100k):Q').title(''),
    color = alt.Color('Jurisdiction:N', scale=alt.Scale(
            domain = ["State", "Tribal Authority"],
            range = ["#FFAE42", "#25e1ff"])))
    full_plot = (density_plot + lines).properties(
        title = "The Average AIAN Drug Related Death Rate is higher in PL280 States"
    )

    return full_plot

pl280_density()

alt.LayerChart(...)

When we take the average AIAN death rate by state, and put all state-year
observations into a density plot, we see that while both death rate distributions
are centered at zero, the mean drug related death rate where States have criminal jurisdiciton
is higher.

In [52]:
#Bubble Plot
def scatter_state():

    df = AIAN_deaths.with_columns(pl.col("population_AIAN").alias("AIAN Population"))
    chart = alt.Chart(df).mark_point(size = 60).encode(
        x = alt.X("drd_non_p100k:Q", title = "Non-AIAN Drug Related Death Rate per 100k" ),
        y = alt.Y("drd_AIAN_p100k:Q", title = "AIAN Drug Related Death Rate per 100k"),
        color = alt.Color('Jurisdiction:N', scale=alt.Scale(
            domain = ["State", "Tribal Authority"],
            range = ["#d58833", "#417ecf"])),
        size = "AIAN Population:Q"
    )

    return chart.properties(title = "Correspondence in the Drug-Related Death Rate for AIAN and non-AIAN persons by State",
                            height = 400,
                            width = 500)

scatter_state()

alt.Chart(...)

One way to control for variation in the spread of the opiods epidemic is to plot
the death rate of non-native populations against the death rate for the native populations.
As expected the correspondence is roughly one-to-one. We are interested in if the different
types of states have a greater than 1:1 or lesser than 1:1 ratio. It appears that most of
the states above the 1:1 line are states that hold criminal jurisdiction on reservations.

In [53]:
# Title seems to appear differently when transform_regression is applied
#Plot Regression
def state_regression_plot():
    df = AIAN_deaths
    chart = alt.Chart(df).mark_point(size = 40, opacity = 0.2).encode(
        x = alt.X("drd_non_p100k:Q", title = "Non-AIAN Drug Related Death Rate per 100k"),
        y = alt.Y("drd_AIAN_p100k:Q", title = "AIAN Drug Related Death Rate per 100k"),
        color = alt.Color('Jurisdiction:N', scale=alt.Scale(
            domain = ["State", "Tribal Authority"],
            range = ["#d58833", "#417ecf"])))
    
    
    full_chart = chart + chart.transform_regression('drd_AIAN_p100k', 'drd_non_p100k',
                                              groupby = ['Jurisdiction']).mark_line()
    
    full_chart = full_chart.properties(title = "Regression Analysis of AIAN Death Rates")

    return full_chart

state_regression_plot()
#Add legend colors for the lines

alt.LayerChart(...)

These lines are regressions in each group where the dependent variable is
the AIAN death rate and the independent variable is the Non-AIAN death rate.
Notable the death rate is higher in states that hold criminal jurisdiction, and
it rises much steeper than the other line. This suggests that as opiods crisis
worsened, we see a larger difference.  

In [54]:
county_drd = pl.read_csv("data/drd_by_county.csv", ignore_errors=True)
counties = alt.topo_feature(data.us_10m.url, 'counties')
county_centroids = pl.read_csv("data/County Centroids.csv")
county_centroids= county_centroids.with_columns(pl.col("cfips").alias("County.Code"))["County.Code", "latitude", "longitude"]
county_drd = county_drd.join(county_centroids, "County.Code", "left")

def pad_code_5(int):
    string = str(int)
    if len(string) < 5:
       string = "".join(["0", string])
    
    return string

def consolidate_counties(state_abb):

    df= county_drd.filter(pl.col("state_abb")==state_abb)
    df = df.with_columns(pl.col("state_crim").map_elements(make_juris_labels).alias("Jurisdiction"),
                         pl.col("County.Code").map_elements(pad_code_5).alias("id"),
                         (pl.col("OD_rate_AIAN")*100000).alias("drd_AIAN_p100k"))
    
    df = df.group_by("id").agg(
            pl.col("drd_AIAN_p100k").mean(),
            pl.col("Jurisdiction").first(),
            pl.col("cnty_name").first(),
            pl.col("latitude").first(),
            pl.col("longitude").first())
    
    return df

#def consolidate_cnty_year(state_abb, year):#
#
#    df= county_drd.filter(pl.col("state_abb")==state_abb)
#    df = df.filter(pl.col("Year")== year)
#    df = df.with_columns(pl.col("state_crim").map_elements(make_juris_labels, pl.).alias("Jurisdiction"),
                       #  pl.col("County.Code").map_elements(pad_code_5).alias("id"),
                      #   (pl.col("OD_rate_AIAN")*100000).alias("drd_AIAN_p100k"))
    
#    return df


In [55]:
states = alt.topo_feature(data.us_10m.url, 'states')
counties = alt.topo_feature(data.us_10m.url, 'counties')
# This source helped me understand how to pass a filter to Js
# https://stackoverflow.com/questions/55336762/filtering-to-state-level-for-choropleth-maps-in-altair 

def map_county(state_abb):

    df = consolidate_counties(state_abb)
    fips_str = str(df["id"][0])[0:2]
    fips_int = int(fips_str)
    df = df.with_columns(pl.col("id").map_elements(int))

    state_outline = alt.Chart(states).mark_geoshape(
        fill='lightgray',
        stroke='white',
        strokeWidth=0.5
    ).transform_filter(
        alt.datum.id == fips_int
    ).project('albersUsa')

    default_value = "No Tribal Lands"
    county_map = alt.Chart(counties).mark_geoshape(
        stroke='white',
        strokeWidth=0.5
    ).encode(
            color = alt.condition(
                alt.datum.value == default_value,
                alt.value('lightgray'),  # Default color
                alt.Color('Jurisdiction:N', scale=alt.Scale(
            domain = ["State", "Tribal Authority"],
            range = ["#d58833", "#417ecf"])))
    ).transform_lookup(
        lookup = 'id',
        from_=alt.LookupData(df, 'id', df.columns),
        default = default_value
    ).transform_calculate(state_id = "(datum.id / 1000)|0"
    ).transform_filter((alt.datum.state_id==fips_int)
    ).project('albersUsa')
    full_map = state_outline + county_map

    bubbles = alt.Chart(df).mark_circle().encode(
    longitude = 'longitude:Q',
    latitude = 'latitude:Q',
    size = alt.Size('drd_AIAN_p100k:Q', title = "AIAN Drug Related Death Rate"),
    color = alt.value("#8A00C4")
    ).project("albersUsa")

    return full_map + bubbles

map_county("MN").properties(title = "Minnesota: AIAN Drug Related Deaths in Counties with Tribal Lands")

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
/var/folders/p9/6789_cm149s8kj127lfbgwy40000gn/T/ipykernel_38282/3820933632.py:11: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(pl.col("id").map_elements(int))


alt.LayerChart(...)

In [56]:
def map_county_year(state_abb, year):

    df = consolidate_cnty_year(state_abb, year)
    fips_str = str(df["id"][0])[0:2]
    fips_int = int(fips_str)
    df = df.with_columns(pl.col("id").map_elements(int))

    state_outline = alt.Chart(states).mark_geoshape(
        fill='lightgray',
        stroke='white',
        strokeWidth=0.5
    ).transform_filter(
        alt.datum.id == fips_int
    ).project('albersUsa')

    default_value = "No Tribal Lands"
    county_map = alt.Chart(counties).mark_geoshape(
        stroke='white',
        strokeWidth=0.5
    ).encode(
            color = alt.condition(
                alt.datum.value == default_value,
                alt.value('lightgray'),  # Default color
                alt.Color('Jurisdiction:N', scale=alt.Scale(
            domain = ["State", "Tribal Authority"],
            range = ["#d58833", "#417ecf"])))
    ).transform_lookup(
        lookup = 'id',
        from_=alt.LookupData(df, 'id', df.columns),
        default = default_value
    ).transform_calculate(state_id = "(datum.id / 1000)|0"
    ).transform_filter((alt.datum.state_id==fips_int)
    ).project('albersUsa')
    full_map = state_outline + county_map

    bubbles = alt.Chart(df).mark_circle().encode(
    longitude = 'longitude:Q',
    latitude = 'latitude:Q',
    size = alt.Size('drd_AIAN_p100k:Q', title = "AIAN Drug Related Death Rate"),
    color = alt.value("#8A00C4")
    ).project("albersUsa")

    return full_map + bubbles

map_county_year("MN", 2020)

NameError: name 'consolidate_cnty_year' is not defined

While each state has legislative provisions about who has criminal jurisdiction on reservation,
in reality many states had carve outs for specific tribes. These carve outs let us examine variation
within state. In both Minnesota and Oregon the Counties where native american reservations have
criminal jurisdiction appear to have lower AIAN death rates. 

In [23]:
map_county("OR").properties(title = "Oregon: AIAN Drug Related Deaths in Counties with Tribal Lands")

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
/var/folders/p9/6789_cm149s8kj127lfbgwy40000gn/T/ipykernel_38282/3820933632.py:11: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(pl.col("id").map_elements(int))


alt.LayerChart(...)

In [57]:
map_county("AK")

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
/var/folders/p9/6789_cm149s8kj127lfbgwy40000gn/T/ipykernel_38282/3820933632.py:11: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(pl.col("id").map_elements(int))


alt.LayerChart(...)

In [ ]:
state_per = round(sum(AIAN_deaths["deaths_AIAN"] == 5) / len(AIAN_deaths["deaths_AIAN"]), 4) * 100
county_per = round(sum(county_drd["deaths_AIAN"] == 5) / len(county_drd["deaths_AIAN"]), 4) * 100
percentage_array = np.array([state_per, county_per])
names_array = np.array(["State Level", "County Level"])

undefined_data = {
    "Level of Data Aggregation": names_array,
    "% of Observations with Supressed Data" : percentage_array
}
undefined_data_df = pd.DataFrame(undefined_data)

def undefined_chart():

    df = undefined_data_df
    chart = alt.Chart(df).mark_bar().encode(
        y = alt.Y("% of Observations with Supressed Data:Q", scale =
                  alt.Scale(domain = [0 , 50])),
        x = alt.X("Level of Data Aggregation:N")

    ).properties(
        width = 200,
        height = 300,
        title = "Deaths Data Aggregated at the County Level Has Almost Twice as Many Undefined Values"
    )
    text = chart.mark_text(
        align='center',
        baseline='middle',
        dy = 8
    ).encode(
        text="% of Observations with Supressed Data:Q"
    )

    return chart + text

undefined_chart()

In [ ]:
def county_regression_plot():
    df = county_drd.filter(pl.col("population_AIAN") < 1000)
    chart = alt.Chart(df).mark_point(size = 40, opacity = 0.2).encode(
        x = alt.X("drd_non_p100k:Q", title = "Non-AIAN Drug Related Death Rate per 100k"),
        y = alt.Y("drd_AIAN_p100k:Q", title = "AIAN Drug Related Death Rate per 100k"),
        color = alt.Color('Jurisdiction:N', scale=alt.Scale(
            domain = ["State", "Tribal Authority"],
            range = ["#FFAE42", "#7c309aa3"])))
    
    df_weighted = df.with_columns((pl.col("drd_AIAN_p100k") *(pl.col("population_AIAN")**(1/2))
     ).alias("weighted_AIAN_drd") )
    
    full_chart = chart + chart.transform_regression('drd_AIAN_p100k', 'drd_non_p100k',
                                              groupby = ['Jurisdiction']).mark_line()
    
    full_chart = full_chart.properties(title = "Controlling for non-AIAN Deaths, State Criminal Jurisdiction Corresponds to Higher AIAN Deaths")

    return full_chart

county_regression_plot()

In [ ]:
def county_regression_weighted_plot():
    df = county_drd
    chart_no_weights = alt.Chart(df).mark_point(size = 40, opacity = 0.2).encode(
        x = alt.X("drd_non_p100k:Q", title = "Non-AIAN Drug Related Death Rate per 100k"),
        y = alt.Y("drd_AIAN_p100k:Q", title = "AIAN Drug Related Death Rate per 100k"),
        color = alt.Color('Jurisdiction:N', scale=alt.Scale(
            domain = ["State", "Tribal Authority"],
            range = ["#FFAE42", "#7c309aa3"])))
    
    df_weighted = df.with_columns((pl.col("drd_AIAN_p100k") *(pl.col("population_AIAN")**(1/2))
     ).alias("weighted_AIAN_drd") )

    chart_weights = alt.Chart(df_weighted).mark_point(size = 40, opacity = 0.2).encode(
    x = alt.X("drd_non_p100k:Q", title = "Non-AIAN Drug Related Death Rate per 100k"),
    y = alt.Y("weighted_AIAN_drd:Q", title = "AIAN Drug Related Death Rate per 100k"),
    color = alt.Color('Jurisdiction:N', scale=alt.Scale(
        domain = ["State", "Tribal Authority"],
        range = ["#FFAE42", "#7c309aa3"])))
    
    full_chart = chart_weights + chart_weights.transform_regression("weighted_AIAN_drd", 'drd_non_p100k',
                                              groupby = ['Jurisdiction']).mark_line()
    
    full_chart = full_chart.properties(title = "Controlling for non-AIAN Deaths, State Criminal Jurisdiction Corresponds to Higher AIAN Deaths")

    return full_chart

county_regression_weighted_plot()